In [ ]:
import sys, os
basedir = ''
if "__file__" in globals(): basedir = os.path.dirname(__file__)
sys.path.append(os.path.join(basedir, os.path.pardir, 'python'))

Then load everything we need from `sz_problem` and also set our default plotting preferences.

In [ ]:
import utils
from sz_base import default_params, allsz_params
from sz_slab import create_slab
from sz_geometry import create_sz_geometry
from sz_problem import SubductionProblem, plot_slab_temperatures
import pyvista as pv
import geometry as geo
import utils
from mpi4py import MPI
import dolfinx as df
import dolfinx.fem.petsc
import numpy as np
import scipy as sp
import ufl
import basix.ufl as bu
import matplotlib.pyplot as pl
import copy
import pyvista as pv
if __name__ == "__main__" and "__file__" in globals():
    pv.OFF_SCREEN = True
import pathlib
output_folder = pathlib.Path(os.path.join(basedir, "output"))
output_folder.mkdir(exist_ok=True, parents=True)

In [ ]:
resscale = 5.0
xs = [0.0, 140.0, 240.0, 400.0]
ys = [0.0, -70.0, -120.0, -200.0]
lc_depth = 40
uc_depth = 15
coast_distance = 0
extra_width = 40
sztype = 'continental'
io_depth_1 = 154 # change to 154 for (at least) case 2
A      = 100.0      # age of subducting slab (Myr)
qs     = 0.065      # surface heat flux (W/m^2)
Vs     = 100.0      # slab speed (mm/yr)

slab = create_slab(xs, ys, resscale, lc_depth)

In [ ]:
# Comparison Case 1 - Steady State Isoviscous - Time Dependent Isoviscous

In [ ]:
geom1 = create_sz_geometry(slab, resscale, sztype, io_depth_1, extra_width, 
                          coast_distance, lc_depth, uc_depth)

sz_case1 = SubductionProblem(geom1, A=A, Vs=Vs, sztype=sztype, qs=qs)
sz_case1.solve_steadystate_isoviscous()

print("Steady State Solution:")
plotter_isoB = utils.plot_scalar(sz_case1.T_i, scale=sz_case1.T0, gather=True, cmap='coolwarm', scalar_bar_args={'title': 'Temperature (deg C)'})
utils.plot_vector_glyphs(sz_case1.vw_i, plotter=plotter_isoB, factor=0.1, gather=True, color='k', scale=utils.mps_to_mmpyr(sz_case1.v0))
utils.plot_vector_glyphs(sz_case1.vs_i, plotter=plotter_isoB, factor=0.1, gather=True, color='k', scale=utils.mps_to_mmpyr(sz_case1.v0))
utils.plot_show(plotter_isoB)

utils.plot_save(plotter_isoB, output_folder / "actualsteadystate.png")

for ft in [1, 5, 10, 15, 20, 50, 75, 100]:
    sz_case1T = SubductionProblem(geom1, A=A, Vs=Vs, sztype=sztype, qs=qs)
    sz_case1T.solve_timedependent_isoviscous(ft, 0.05, theta=0.5)

    Tdiff = df.fem.Function(sz_case1.V_T)
    Tdiff.x.array[:] = sz_case1.T_i.x.array - sz_case1T.T_i.x.array
    plotter_iso = utils.plot_scalar(Tdiff, gather=True, cmap='coolwarm', scalar_bar_args={'title': 'difference'})
    plotter_isoA = utils.plot_scalar(sz_case1T.T_i, scale=sz_case1T.T0, gather=True, cmap='coolwarm', scalar_bar_args={'title': 'Temperature (deg C)'})
    
    utils.plot_vector_glyphs(sz_case1T.vw_i, plotter=plotter_isoA, factor=0.1, gather=True, color='k', scale=utils.mps_to_mmpyr(sz_case1T.v0))
    utils.plot_vector_glyphs(sz_case1T.vs_i, plotter=plotter_isoA, factor=0.1, gather=True, color='k', scale=utils.mps_to_mmpyr(sz_case1T.v0))

    print("Case (Myo): {ft}".format(ft=ft))
    utils.plot_show(plotter_iso) # plot difference
    utils.plot_show(plotter_isoA) # plot actual solution
    
    utils.plot_save(plotter_iso, output_folder / "case1-{:d}.png".format(ft,))
    utils.plot_save(plotter_isoA, output_folder / "actual1-{:d}.png".format(ft,))

In [ ]:
# Comparison Case 2 - Steady State Dislocation Creep - Time Dependent Dislocation Creep

In [ ]:
geom2 = create_sz_geometry(slab, resscale, sztype, io_depth_1, extra_width, 
                          coast_distance, lc_depth, uc_depth)

sz_case2 = SubductionProblem(geom2, A=A, Vs=Vs, sztype=sztype, qs=qs)
sz_case2.solve_steadystate_dislocationcreep()

print("Steady State Solution:")
plotter_isoB = utils.plot_scalar(sz_case2.T_i, scale=sz_case2.T0, gather=True, cmap='coolwarm', scalar_bar_args={'title': 'Temperature (deg C)', 'bold':True})
utils.plot_vector_glyphs(sz_case2.vw_i, plotter=plotter_isoB, factor=0.1, gather=True, color='k', scale=utils.mps_to_mmpyr(sz_case2.v0))
utils.plot_vector_glyphs(sz_case2.vs_i, plotter=plotter_isoB, factor=0.1, gather=True, color='k', scale=utils.mps_to_mmpyr(sz_case2.v0))
utils.plot_geometry(geom2, plotter=plotter_isoB, color='green', width=2)
utils.plot_couplingdepth(slab, plotter=plotter_isoB, render_points_as_spheres=True, point_size=10.0, color='green')
utils.plot_show(plotter_isoB)

utils.plot_save(plotter_isoB, output_folder / "actualsteadystate.png", transparent_background=True)

for ft in [1, 5, 20, 50, 100]:
    sz_case2T = SubductionProblem(geom2, A=A, Vs=Vs, sztype=sztype, qs=qs)
    sz_case2T.solve_timedependent_dislocationcreep(ft, 0.05, theta=0.5, verbosity=1)

    Tdiff = df.fem.Function(sz_case2.V_T)
    Tdiff.x.array[:] = sz_case2.T_i.x.array - sz_case2T.T_i.x.array
    plotter_iso = utils.plot_scalar(Tdiff, gather=True, cmap='coolwarm', scalar_bar_args={'title': 'Temperature difference (deg C)', 'bold':True})
    plotter_isoA = utils.plot_scalar(sz_case2T.T_i, scale=sz_case2T.T0, gather=True, cmap='coolwarm', scalar_bar_args={'title': 'Temperature (deg C)', 'bold':True})
    
    utils.plot_vector_glyphs(sz_case2T.vw_i, plotter=plotter_isoA, factor=0.1, gather=True, color='k', scale=utils.mps_to_mmpyr(sz_case2T.v0))
    utils.plot_vector_glyphs(sz_case2T.vs_i, plotter=plotter_isoA, factor=0.1, gather=True, color='k', scale=utils.mps_to_mmpyr(sz_case2T.v0))

    
    utils.plot_geometry(geom2, plotter=plotter_iso, color='green', width=2)
    utils.plot_couplingdepth(slab, plotter=plotter_iso, render_points_as_spheres=True, point_size=10.0, color='green')
    utils.plot_geometry(geom2, plotter=plotter_isoA, color='green', width=2)
    utils.plot_couplingdepth(slab, plotter=plotter_isoA, render_points_as_spheres=True, point_size=10.0, color='green')

    print("Case (Myo): {ft}".format(ft=ft))
    utils.plot_show(plotter_iso) # plot difference
    utils.plot_show(plotter_isoA) # plot actual solution

    utils.plot_save(plotter_iso, output_folder / "case1-{:d}.png".format(ft,), transparent_background=True)
    utils.plot_save(plotter_isoA, output_folder / "actual1-{:d}.png".format(ft,), transparent_background=True)

In [ ]:
# Comparison Case 3 - Steady State Isoviscous - Steady State Dislocation Creep

In [ ]:
geom3 = create_sz_geometry(slab, resscale, sztype, io_depth_1, extra_width, 
                          coast_distance, lc_depth, uc_depth)

sz_case3 = SubductionProblem(geom3, A=A, Vs=Vs, sztype=sztype, qs=qs)
sz_case3.solve_steadystate_isoviscous()

for ft in [1, 5, 10, 15, 20, 50, 75, 100]:
    sz_case3T = SubductionProblem(geom3, A=A, Vs=Vs, sztype=sztype, qs=qs)
    sz_case3T.solve_steadystate_dislocationcreep()

    Tdiff = df.fem.Function(sz_case3.V_T)
    Tdiff.x.array[:] = sz_case3.T_i.x.array - sz_case3T.T_i.x.array
    plotter_iso = utils.plot_scalar(Tdiff, gather=True, cmap='coolwarm', scalar_bar_args={'title': 'difference'})
    plotter_isoA = utils.plot_scalar(sz_case3T.T_i, scale=sz_case3T.T0, gather=True, cmap='coolwarm', scalar_bar_args={'title': 'Temperature (deg C)'})

    utils.plot_vector_glyphs(sz_case3T.vw_i, plotter=plotter_isoA, factor=0.1, gather=True, color='k', scale=utils.mps_to_mmpyr(sz_case3T.v0))
    utils.plot_vector_glyphs(sz_case3T.vs_i, plotter=plotter_isoA, factor=0.1, gather=True, color='k', scale=utils.mps_to_mmpyr(sz_case3T.v0))
    
    utils.plot_show(plotter_iso3)
    utils.plot_save(plotter_iso3, output_folder / "sz_comparisons.png")

In [ ]:
# Comparison Case 4 - Time Dependent Isoviscous - Time Dependent Dislocation Creep

In [ ]:
geom4 = create_sz_geometry(slab, resscale, sztype, io_depth_1, extra_width, 
                          coast_distance, lc_depth, uc_depth)

for ft in [1, 100]:
    sz_case4 = SubductionProblem(geom4, A=A, Vs=Vs, sztype=sztype, qs=qs)
    sz_case4T = SubductionProblem(geom4, A=A, Vs=Vs, sztype=sztype, qs=qs)
    
    sz_case4.solve_timedependent_isoviscous(ft, 0.05, theta=0.5)
    sz_case4T.solve_timedependent_dislocationcreep(ft, 0.05, theta=0.5)

    Tdiff = df.fem.Function(sz_case4.V_T)
    Tdiff.x.array[:] = sz_case4.T_i.x.array - sz_case4T.T_i.x.array
    plotter_iso = utils.plot_scalar(Tdiff, gather=True, cmap='coolwarm', scalar_bar_args={'title': 'difference'})
    plotter_isoA = utils.plot_scalar(sz_case4.T_i, scale=sz_case4.T0, gather=True, cmap='coolwarm', scalar_bar_args={'title': 'Temperature (deg C)'})
    plotter_isoB = utils.plot_scalar(sz_case4T.T_i, scale=sz_case4T.T0, gather=True, cmap='coolwarm', scalar_bar_args={'title': 'Temperature (deg C)'})

    utils.plot_vector_glyphs(sz_case4.vw_i, plotter=plotter_isoA, factor=0.1, gather=True, color='k', scale=utils.mps_to_mmpyr(sz_case4.v0))
    utils.plot_vector_glyphs(sz_case4.vs_i, plotter=plotter_isoA, factor=0.1, gather=True, color='k', scale=utils.mps_to_mmpyr(sz_case4.v0))

    utils.plot_vector_glyphs(sz_case4T.vw_i, plotter=plotter_isoB, factor=0.1, gather=True, color='k', scale=utils.mps_to_mmpyr(sz_case4T.v0))
    utils.plot_vector_glyphs(sz_case4T.vs_i, plotter=plotter_isoB, factor=0.1, gather=True, color='k', scale=utils.mps_to_mmpyr(sz_case4T.v0))
    
    print("Case (Myo): {ft} Temp Difference:".format(ft=ft))
    utils.plot_show(plotter_iso) # plot difference
    print("Case (Myo): {ft} Actual Solution, Time Dependent Isoviscous".format(ft=ft))
    utils.plot_show(plotter_isoA) # plot actual solution for time dependent isoviscous
    print("Case (Myo): {ft} Actual Solution, Time Dependent Dislocation Creep".format(ft=ft))
    utils.plot_show(plotter_isoB) # plot actual solution for time dependent dislocation creep

    utils.plot_save(plotter_iso, output_folder / "case1-{:d}.png".format(ft,))
    utils.plot_save(plotter_isoA, output_folder / "actual1-{:d}.png".format(ft,))
    utils.plot_save(plotter_isoB, output_folder / "actualDC-{:d}.png".format(ft,))